In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
pv_df = pd.read_csv('../raw_data/1980-2022_pv.csv')
pv_df.head(2)

,Unnamed: 0.1,Unnamed: 0,local_time,electricity,irradiance_direct,irradiance_diffuse,temperature,source
0,0,315532800000,1980-01-01 01:00:00+01:00,0.0,0.0,0.0,-1.296,data/pv_data/1980_pv.csv
1,1,315536400000,1980-01-01 02:00:00+01:00,0.0,0.0,0.0,-1.216,data/pv_data/1980_pv.csv


In [40]:
x = pv_df.loc[0,'local_time']
print('x: ', x)
#dt.datetime.strptime("21 June, 2018", "%d %B, %Y")
dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z")

x:  1980-01-01 01:00:00+01:00


datetime.datetime(1980, 1, 1, 1, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600)))

In [41]:
# load data
pv_df = pd.read_csv('../raw_data/1980-2022_pv.csv')

# drop columns
pv_df.drop(columns=['irradiance_direct','irradiance_diffuse','temperature',
                    'source','Unnamed: 0.1'], inplace=True)

# convert dtypes
pv_df.electricity = pv_df.electricity.astype(float)

#pv_df.local_time    = pd.to_datetime(pv_df.local_time) #
pv_df.local_time    = pv_df.local_time.apply(lambda x:
                                            dt.datetime.strptime(x,
                                            "%Y-%m-%d %H:%M:%S%z"))


pv_df['Unnamed: 0'] = pd.to_datetime(pv_df['Unnamed: 0'], unit='ms').dt.tz_localize('UTC')

# name columns
pv_df.rename(columns={'Unnamed: 0': 'utc_time'}, inplace=True)

# show
pv_df.head(2)

,utc_time,local_time,electricity
0,1980-01-01 00:00:00+00:00,1980-01-01 01:00:00+01:00,0.0
1,1980-01-01 01:00:00+00:00,1980-01-01 02:00:00+01:00,0.0


In [7]:
pv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376944 entries, 0 to 376943
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   utc_time     376944 non-null  datetime64[ns, UTC]
 1   local_time   376944 non-null  object             
 2   electricity  376944 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(1)
memory usage: 8.6+ MB


In [21]:
pv_df.tail(2)

,utc_time,local_time,electricity
376942,2022-12-31 22:00:00+00:00,2022-12-31 23:00:00+01:00,0.0
376943,2022-12-31 23:00:00+00:00,2023-01-01 00:00:00+01:00,0.0


In [25]:
# unique dates match the df length
print('pv_df number of rows: ', pv_df.shape[0])
print('Number of unique values:')
pv_df.nunique()

pv_df number of rows:  376944
Number of unique values:


utc_time       376944
local_time     376944
electricity       892
dtype: int64

In [26]:
# There are no NA values
pv_df.isna().sum()

utc_time       0
local_time     0
electricity    0
dtype: int64

In [27]:
print('Number of negative electricity values: ', len(pv_df[pv_df.electricity < 0.0]))
print('Number of electricity values above max. capacity (1 kw): ', len( pv_df[pv_df.electricity > 1.0]))

Number of negative electricity values:  0
Number of electricity values above max. capacity (1 kw):  0


In [28]:
def get_pv_data() -> pd.DataFrame:
    """
    Load raw data from local directory
    """
    df = pd.read_csv('../raw_data/1980-2022_pv.csv')
    print('# data loaded')
    return df


In [43]:
def clean_pv_data(pv_df: pd.DataFrame) ->pd.DataFrame:
    """
    Remove unnecessary columns and convert to right dtypes
    """
    # remove unnevessary columns
    pv_df.drop(columns=['irradiance_direct','irradiance_diffuse','temperature',
                    'source','Unnamed: 0.1'], inplace=True)

    # convert dtypes
    pv_df.electricity = pv_df.electricity.astype(float)

    pv_df.local_time    = pv_df.local_time.apply(lambda x:
                                            dt.datetime.strptime(x,
                                            "%Y-%m-%d %H:%M:%S%z")) # pd.to_datetime gives warning

    pv_df['Unnamed: 0'] = pd.to_datetime(pv_df['Unnamed: 0'],
                                         unit='ms').dt.tz_localize('UTC')
    # correct column names
    pv_df.rename(columns={'Unnamed: 0': 'utc_time'}, inplace=True)

    print('# data cleaned')
    return pv_df

In [44]:
df = get_pv_data()
df = clean_pv_data(df)
df.head()

# data cleaned


,utc_time,local_time,electricity
0,1980-01-01 00:00:00+00:00,1980-01-01 01:00:00+01:00,0.0
1,1980-01-01 01:00:00+00:00,1980-01-01 02:00:00+01:00,0.0
2,1980-01-01 02:00:00+00:00,1980-01-01 03:00:00+01:00,0.0
3,1980-01-01 03:00:00+00:00,1980-01-01 04:00:00+01:00,0.0
4,1980-01-01 04:00:00+00:00,1980-01-01 05:00:00+01:00,0.0
